In [ ]:
import paho.mqtt.client as mqtt
import ssl
import json
import tempfile
import numpy as np
import pandas as pd
import joblib

# ===== Model and LabelEncoder Upload =====
model = joblib.load("stress_rf_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# ===== MQTT and certificate settings =====
BROKER = "zd67ac9c.ala.eu-central-1.emqxsl.com"
PORT = 8883
TOPIC_SUB = "stress_detection/subject01"
TOPIC_PUB = "stress_detection/results"
USERNAME = "emqx"
PASSWORD = "mqtt1234"

ROOT_CA = """-----BEGIN CERTIFICATE-----
MIIDrzCCApegAwIBAgIQCDvgVpBCRrGhdWrJWZHHSjANBgkqhkiG9w0BAQUFADBh
MQswCQYDVQQGEwJVUzEVMBMGA1UEChMMRGlnaUNlcnQgSW5jMRkwFwYDVQQLExB3
d3cuZGlnaWNlcnQuY29tMSAwHgYDVQQDExdEaWdpQ2VydCBHbG9iYWwgUm9vdCBD
QTAeFw0wNjExMTAwMDAwMDBaFw0zMTExMTAwMDAwMDBaMGExCzAJBgNVBAYTAlVT
MRUwEwYDVQQKEwxEaWdpQ2VydCBJbmMxGTAXBgNVBAsTEHd3dy5kaWdpY2VydC5j
b20xIDAeBgNVBAMTF0RpZ2lDZXJ0IEdsb2JhbCBSb290IENBMIIBIjANBgkqhkiG
9w0BAQEFAAOCAQ8AMIIBCgKCAQEA4jvhEXLeqKTTo1eqUKKPC3eQyaKl7hLOllsB
CSDMAZOnTjC3U/dDxGkAV53ijSLdhwZAAIEJzs4bg7/fzTtxRuLWZscFs3YnFo97
nh6Vfe63SKMI2tavegw5BmV/Sl0fvBf4q77uKNd0f3p4mVmFaG5cIzJLv07A6Fpt
43C/dxC//AH2hdmoRBBYMql1GNXRor5H4idq9Joz+EkIYIvUX7Q6hL+hqkpMfT7P
T19sdl6gSzeRntwi5m3OFBqOasv+zbMUZBfHWymeMr/y7vrTC0LUq7dBMtoM1O/4
gdW7jVg/tRvoSSiicNoxBN33shbyTApOB6jtSj1etX+jkMOvJwIDAQABo2MwYTAO
BgNVHQ8BAf8EBAMCAYYwDwYDVR0TAQH/BAUwAwEB/zAdBgNVHQ4EFgQUA95QNVbR
TLtm8KPiGxvDl7I90VUwHwYDVR0jBBgwFoAUA95QNVbRTLtm8KPiGxvDl7I90VUw
DQYJKoZIhvcNAQEFBQADggEBAMucN6pIExIK+t1EnE9SsPTfrgT1eXkIoyQY/Esr
hMAtudXH/vTBH1jLuG2cenTnmCmrEbXjcKChzUyImZOMkXDiqw8cvpOp/2PV5Adg
06O/nVsJ8dWO41P0jmP6P6fbtGbfYmbW0W5BjfIttep3Sp+dWOIrWcBAI+0tKIJF
PnlUkiaY4IBIqDfv8NZ5YBberOgOzW6sRBc4L0na4UU+Krk2U886UAb3LujEV0ls
YSEY1QSteDwsOoBrp+uvFRTp2InBuThs4pFsiv9kuXclVzDAGySj4dzp30d8tbQk
CAUw7C29C79Fv1C5qfPrmAESrciIxpg0X40KPMbp1ZWVbd4=
-----END CERTIFICATE-----"""

# Save certificate as temporary file
with tempfile.NamedTemporaryFile(delete=False, mode='w', suffix='.pem') as f:
    f.write(ROOT_CA)
    ca_path = f.name

# ===== MQTT Message Function =====
def on_message(client, userdata, msg):
    try:
        data = json.loads(msg.payload.decode())
        print("\n📥 Received MQTT Data:")
        print(json.dumps(data, indent=2))

        # 60 samples are expected
        samples = data.get("samples", [])
        if len(samples) != 60:
            print(f"❗ Warning: 60 data was expected, but {len(samples)} data arrived.
            return

        # Convert to DataFrame and create average vector
        df_samples = pd.DataFrame(samples)
        mean_vector = df_samples[['temperature', 'humidity', 'gsr', 'pulse']].mean().values.reshape(1, -1)

        # Model predict
        prediction_encoded = model.predict(mean_vector)[0]
        prediction_label = label_encoder.inverse_transform([prediction_encoded])[0]

        print(f"🔮 Predict: {prediction_label}")

        #Publish
        result_payload = json.dumps({
            "timestamp": data.get("timestamp"),
            "prediction": prediction_label
        })
        client.publish(TOPIC_PUB, result_payload)
        print(f"📤 Prediction MQTT broadcast done: {TOPIC_PUB}")

    except Exception as e:
        print("❌ Error:", e)

# ===== MQTT SETTINGS =====
client = mqtt.Client()
client.username_pw_set(USERNAME, PASSWORD)
client.tls_set(ca_certs=ca_path, certfile=None, keyfile=None, cert_reqs=ssl.CERT_REQUIRED,
               tls_version=ssl.PROTOCOL_TLSv1_2)

client.on_message = on_message

# ===== Connect and Start Listening =====
client.connect(BROKER, PORT)
client.subscribe(TOPIC_SUB)
print(f"📡 Listening: {TOPIC_SUB}")
client.loop_forever()
